# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [ ]:
# Install Strands using pip

!pip install strands-agents strands-agents-tools


## Creating Your Music Composer Agent

Lets put in our AWS credentials, and ensure that it works!


### Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.5 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [ ]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


## Lets Build a Task-Specific Agent - Music Composer Bot 🍽️

Duck Duck Go is a website that is used to search for financial information, for example: yahoo finance.

Lets create a practical example of a task-specific agent. We create a `MusicBot` that makes music or songs. Lets dive in !!

Here's what we will create :

In [ ]:
# Install the required packages
%pip install duckduckgo-search # Also install strands-agents strands-agents-tools if you haven't already

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.8 MB/s eta 0:00:00


In [ ]:
from strands import Agent, tool
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException, DuckDuckGoSearchException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [ ]:
# Create a music composing assistant
music_agent = Agent(
    system_prompt="""You're a creative lyricist and genre blender. Take lyrical and stylistic inspiration from the following songs:

- **Song A** (Rock + Motivational): Themes of rebellion, grit, and triumph. Think power chords, anthemic choruses, and lines like “I rise with every scar.”
- **Song B** (Happy + Upbeat): Sunshine-drenched nostalgia and playful love. Use warm, cheeky imagery like “Dancing in coffee stains” with a bouncy acoustic rhythm.
- **Song C** (Bloody Sweet): Intoxicating love and poetic vulnerability. Dreamy synth textures, bittersweet metaphors such as “You stitched my veins with candy strings.”

Your task is to use duckduckgo and go to YouTube or Soundcloud or any opensource music databases to:

1. Extract the emotional themes, lyrical motifs, and musical moods from these styles.
2. Synthesize them into a single, cohesive song titled **“Neon Scars & Honey Rain”**.
3. Structure the lyrics as:
   - Verses that weave grit and romantic tension
   - A chorus that bursts with euphoric release
   - An optional bridge that reflects inner resolve and transformation

Keep all lyrics original. Use poetic techniques like metaphor, ABAB rhyme, or free verse. Embrace emotional layering — it should feel like falling in love while marching through fire under summer skies.

The final result should read like a song that blends **soft rock**, **k-pop**, and **electronic music**.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [ ]:
response = music_agent("I am a music enthusiast. I want to create song lyrics that grabs my audience's attention in a good way, and makes them vibe with it. The song lyrics should make sense and have my audience want to see it again. Try to make it similar and unique compared to the Eyes like Fire (Nice Soft Rock theme) by Mister Sommer in YouTube. Can you format the information you give me to align with what I would do if I were to create a song?")

print(response)

# Creating Your Soft Rock Song: "Burning Horizons"

## SONG CONCEPT OVERVIEW
Creating a song similar to soft rock themes with fire imagery requires a balance of emotional depth and accessible lyrics. Your song should evoke feelings while remaining relatable.

## LYRICAL THEME
- **Central metaphor**: Fire as passion/inner strength
- **Emotional tone**: Reflective yet hopeful
- **Narrative arc**: Journey from uncertainty to self-discovery

## SONG STRUCTURE

### Verse 1
```
Silent roads stretch out ahead
Shadows follow where I tread
Mountains rise in morning haze
Memories of better days
Something calling from within
A spark that's waiting to begin
```

### Pre-Chorus
```
And I've been wandering too long
Looking for where I belong
```

### Chorus
```
When horizons burn with golden light
And the darkness starts to fade
I can see beyond my doubts and fears
To the promise of new days
With eyes like fire, I face the dawn
No longer lost, no longer torn
In this moment I am reborn
```

### Verse

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7a4cf2141550>


#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).